In [1]:
import os
import torch
import numpy as np
import pandas as pd
import sys
import random
import h5py
from torch.utils.tensorboard import SummaryWriter

sys.path.append('../')

from models.mcnnpytorch.src.crowd_count import CrowdCounter
from models.mcnnpytorch.src import network
from models.mcnnpytorch.src.data_loader import ImageDataLoader
from models.mcnnpytorch.src.timer import Timer
from models.mcnnpytorch.src import utils
from models.mcnnpytorch.src.evaluate_model import evaluate_model

In [12]:
try:
    from termcolor import cprint
except ImportError:
    cprint = None


def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)

In [2]:
# Directory Configurations

method = 'mcnn'
dataset_name = 'shtechB'
output_dir = '../output/mcnnpytorch/saved_models'

train_path = '../data/SHT/part_B_final/train_data/consolidated'
train_gt_path = '../data/SHT/part_B_final/train_data/ground_truth'
val_path = '../data/SHT/part_B_final/test_data/consolidated'
val_gt_path = '../data/SHT/part_B_final/test_data/ground_truth'

In [3]:
# Create output directory if it doesnt exist

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [11]:
# load model

net = CrowdCounter()
network.weights_normal_init(net, dev=0.01)
net.train()

CrowdCounter(
  (DME): MCNN(
    (branch1): Sequential(
      (0): Conv2d(
        (conv): Conv2d(3, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
        (relu): ReLU(inplace=True)
      )
      (1): Conv2d(
        (conv): Conv2d(16, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (2): Conv2d(
        (conv): Conv2d(32, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (3): Conv2d(
        (conv): Conv2d(16, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
    )
    (branch2): Sequential(
      (0): Conv2d(
        (conv): Conv2d(3, 20, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (1): Conv2d(
        (conv): Conv2d(20, 40, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (relu): ReLU(inplace=True)
      )
      (2): Conv2d(
        (conv): Conv2d(40, 20, 

In [5]:
#training configuration

start_step = 0
end_step = 20
momentum = 0.9
disp_interval = 2
log_interval = 250

train_loss = 0
step_cnt = 0
re_cnt = False
t = Timer()
t.tic()

In [6]:
# Hyperparameters

learning_rate = 0.00001

params = list(net.parameters())
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=learning_rate)



In [7]:
#Tensorboard  config
use_tensorboard = True

writer = SummaryWriter(f'../output/tensorboard/runs')


In [8]:
rand_seed = 64678  
if rand_seed is not None:
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)
    torch.cuda.manual_seed(rand_seed)

In [9]:
# Load the data
data_loader = ImageDataLoader(train_path, train_gt_path, shuffle=True, pre_load=True)
data_loader_val = ImageDataLoader(val_path, val_gt_path, shuffle=False, pre_load=True)
best_mae = sys.maxsize

Pre-loading the data. This may take a while...
Completed Loading 3 files
Pre-loading the data. This may take a while...
Completed Loading 3 files


In [ ]:
for epoch in range(start_step, end_step+1):    
    step = -1
    train_loss = 0
    for blob in data_loader:                
        step = step + 1        
        im_data = blob['data']
        gt_data = blob['gt_density']
        
        density_map = net(im_data, gt_data)
        loss = net.loss
        train_loss += loss.data
        
        # Write to tensorboard
        writer.add_scalar("Loss/train", train_loss, epoch)
        
        step_cnt += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % disp_interval == 0:            
            duration = t.toc(average=False)
            fps = step_cnt / duration
            gt_count = np.sum(gt_data)    
            density_map = density_map.data.cpu().numpy()
            et_count = np.sum(density_map)
            utils.save_results(im_data,gt_data,density_map, output_dir)
            log_text = 'epoch: %4d, step %4d, Time: %.4fs, gt_cnt: %4.1f, et_cnt: %4.1f' % (epoch,
                step, 1./fps, gt_count,et_count)
            log_print(log_text, color='green', attrs=['bold'])
            re_cnt = True    
    
        if re_cnt:                                
            t.tic()
            re_cnt = False

    if (epoch % 2 == 0):
        save_name = os.path.join(output_dir, '{}_{}_{}.h5'.format(method,dataset_name,epoch))
        network.save_net(save_name, net)   
        
        #calculate error on the validation dataset 
        mae,mse = evaluate_model(save_name, data_loader_val)
        if mae < best_mae:
            best_mae = mae
            best_mse = mse
            best_model = '{}_{}_{}.h5'.format(method,dataset_name,epoch)
    
        log_text = 'EPOCH: %d, MAE: %.1f, MSE: %0.1f' % (epoch,mae,mse)
        log_print(log_text, color='green', attrs=['bold'])
        log_text = 'BEST MAE: %0.1f, BEST MSE: %0.1f, BEST MODEL: %s' % (best_mae,best_mse, best_model)
        log_print(log_text, color='green', attrs=['bold'])
        
        if use_tensorboard:
            writer.add_scalar("MAE", mae, epoch)
            writer.add_scalar("MSE", train_loss, epoch)
            writer.add_scalar("train_loss", train_loss/data_loader.get_num_samples(), epoch)


epoch:    0, step    0, Time: 426.2203s, gt_cnt: 230.0, et_cnt:  7.7
epoch:    0, step    2, Time: 8.6151s, gt_cnt: 206.0, et_cnt:  7.5


../models/mcnnpytorch/src/network.py:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  v = Variable(torch.as_tensor(x).type(dtype), requires_grad = False, volatile = True)


EPOCH: 0, MAE: 113.0, MSE: 132.9
BEST MAE: 113.0, BEST MSE: 132.9, BEST MODEL: mcnn_shtechB_0.h5
epoch:    1, step    0, Time: 6.7408s, gt_cnt: 234.0, et_cnt:  5.7
epoch:    1, step    2, Time: 4.9336s, gt_cnt: 230.0, et_cnt:  7.7
epoch:    2, step    0, Time: 2.7558s, gt_cnt: 230.0, et_cnt:  7.7
epoch:    2, step    2, Time: 3.8162s, gt_cnt: 206.0, et_cnt:  7.5
EPOCH: 2, MAE: 113.0, MSE: 132.9
BEST MAE: 113.0, BEST MSE: 132.9, BEST MODEL: mcnn_shtechB_0.h5
epoch:    3, step    0, Time: 2.9597s, gt_cnt: 206.0, et_cnt:  7.5
epoch:    3, step    2, Time: 2.3137s, gt_cnt: 230.0, et_cnt:  7.7
epoch:    4, step    0, Time: 1.2125s, gt_cnt: 230.0, et_cnt:  7.7
epoch:    4, step    2, Time: 2.6159s, gt_cnt: 206.0, et_cnt:  7.5
EPOCH: 4, MAE: 113.0, MSE: 132.9
BEST MAE: 113.0, BEST MSE: 132.9, BEST MODEL: mcnn_shtechB_0.h5
epoch:    5, step    0, Time: 2.0531s, gt_cnt: 206.0, et_cnt:  7.5
epoch:    5, step    2, Time: 1.9633s, gt_cnt: 230.0, et_cnt:  7.7
epoch:    6, step    0, Time: 1.0681s, 

In [ ]:
# Close tensorboard
writer.close()